# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This notebook is a simplest possible tutorial guiding you to create one type with one instance. Have a look at the **chatbot_setup_advanced.jpynp** for more advanced features.

### 1. Preparation

##### 1.1. Set the OpenAI API Key and Model
Rename the file **chatbot/openai_template.py** to **chatbot/openai.py** and set the following keys.
- OPENAI_KEY = "your OpenAI API key in quotes"
- OPENAI_MODEL = "model name in quotes, e.g. gpt-3.5-turbo-16k"

##### 1.2. Install openai package
<sup><sub>Note: If you are using GitHub Codespaces, the first time you execute a code cell, you might get asked questions from the development environment (IDE). You can say yes to recommended installations, and to installing/enabling reecommended extensions. When asked to **Select Another Kernel**, choose **Python Environments** and select the recommende environment, e.g., **Python 3.10.13**</sub></sup>

In [15]:
%pip install openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


##### 1.3. Import Chatbot library

In [16]:
# importiert die Chatbot-Klasse aus dem Modul chatbot.chatbot. 
# Charbot-Klasse ist die hauptschnittstelle, welche die LM-Kommunikation und Datenverwaltung regelt 
from chatbot.chatbot import Chatbot

### 2. Create a chatbot of type **health coach** for user **Daniel Müller**

##### 2.1. Prompt Engineering
To create a chatbot, we need to provide prompts. There are three prompts that can be used to customise the behaviour of the chatbot: a ***role***, ***context***, and ***starter*** prompt.

**Role Prompt (type_role)**

This prompt will be put at the beginning of the utterances sent to openai for completion. If you create multiple chatbots of the same type, this prompt will be the same for each one of them.

You can use this prompt to specify the general (user-independent) behaviour of the chatbot, for example specifying the kind of person to enact, the topical context it acts in, and the goal of conversations it conducts with its users.

In [17]:
# definiert Rolle und das Verhalten des chatbots (Rollenzuweisung, Aufgabe und Kommunkationsstil)
# steuert wie der Chatbot in Interaktionen reagiert , mit welchen Ton 
my_type_role = """
    As a digital therapy coach, check in daily with your patient to assess their well-being related to their chronic condition.
    Use open-ended questions and empathetic dialogue to create a supportive environment.
    Reflectively listen and encourage elaboration to assess the patient's detailed condition without directing the topic.
"""

**Context Prompt (instance_context)**

This prompt will be put right after the role prompt. If you create multiple chatbots of the same type, this prompt will be used only for one of them. <br />

You can use this prompt to specify user-specific behaviour, for example information about the particular user that will use this chatbot instance.

In [18]:
# definiert Kontext (Infos über Benutzer, mit dem er interagiet)
# damit kann er personalisierte und relevante Antworten geben (Interaktionstiefe)
my_instance_context = """
    Meet Daniel Müller, 52, who is tackling obesity with a therapy plan that includes morning-to-noon intermittent fasting, 
    thrice-weekly 30-minute swims, and a switch to whole grain bread.
"""

**Starter Prompt (instance_starter)**

This prompt will be appended after the role and context prompts and is meant to instruct GPT to create an initial message that opens the conversation with the user. <br />

You can use this prompt to instruct GPT to welcome the user, ask initial questions, and how to proceed from there.

In [19]:
# legt ton udn struktur des ersten Gesprächsbeitrags fest 
# Konversation mit Daniel wird initiativ und passend zur Situation vom Chatbot gestartet
my_instance_starter = """
    After reviewing the patient's profile, compose a single, very short message that the therapy coach would use to initiate today's check-in conversation with Daniel.
"""

##### 2.2. Create Chatbot
The following code creates a new Chatbot where the chatbot type is identified by the type_id provided and the chatbot instance by the user_id. These two IDs will be used further below to construct the URL pointing to this chatbot once it is deployed.

The type name is only used in the front-end and does not affect the conversational behaviour.

Once this code is executed, the chatbot is stored in the database.

1. *Die chatbot.db dient als persistent layer, der alle notwendigen Daten speichert, damit der Chatbot funktional bleibt, zwischen Instanzen unterscheidet und Benutzerinteraktionen nachvollziehen kann.*
2. *Man kann type_id und user_id selbst bestimmen, solange sie eindeutig und logisch im Kontext der Anwendung sind.*



In [20]:
bot = Chatbot(  
    # Initialisiert eine neue Chatbot-Instanz
    database_file="database/chatbot.db",  # Gibt den Pfad zur SQLite-Datenbank an, in der die Chatbot-Daten gespeichert werden
    type_id="c48a13d7",  # Eindeutige ID für den Chatbot-Typ (z. B. "Health Coach")
    user_id="fe6d944d",  # Eindeutige ID für den Benutzer, der mit dem Chatbot interagiert
    type_name="Health Coach",  # Name des Chatbot-Typs, hier "Health Coach"
    type_role=my_type_role,  # Definiert die Rolle des Chatbots (aus vorher definiertem Prompt my_type_role)
    instance_context=my_instance_context,  # Kontext für diese spezifische Chatbot-Instanz (z. B. Benutzerinformationen)
    instance_starter=my_instance_starter  # Anweisung, wie der Chatbot das Gespräch beginnen soll
)

##### 2.3. Initiate Conversation Starter (Optional)

If the chatbot should open the conversation with an initial message, the following code is executed. This code will execute a request to GPT, with the role, context and startet prompts specified above. The response from GPT (the initial message) will be stored in the database and therefore the user will see the opening message as soon as they access the chatbot.

###### *Zweck:*
1. *Testen und Debuggen: Es wird überprüft, ob der Chatbot korrekt startet und die erwartete erste Nachricht generiert.*
2. *Einstieg in die Konversation: Die Startnachricht des Chatbots wird bereitgestellt, um das Gespräch mit dem Benutzer zu beginnen.*

Kurz: Der Code zeigt an, wie der Chatbot auf Grundlage der definierten Rolle und des Kontextes die erste Konversationsnachricht erstellt. 

1. Zuerst wird die start()-Methode des Chatbots aufgerufne, welche den *instance_starter* verwendet, um den Einstiegspuntk (erste Nachricht) der Konversation zu erstellen. 
2. Der Chatbot sendet die definierten Prompts (Rolle, Kontext und Starter) an das GPT-Modell über die OpenAI-API. => GPT generiert darauf basierend dann eine Antwort.
3. Die Antwort von GPT wir dann in kleine Teile aufgespalten, in die DB geschrieben (mit der MEthode _append_assistant)
4. Die Methode gibt die generieten Antworten als Lsite ovn Strings zurück ([Guten Morgen Danie, -... ]) => mit print() kann die Liste der Startnachrichten zurückgegeben wreden.

In [21]:
print(bot.start())

['Hi Daniel! How are you today? How are you finding your therapy plan so far?']


### 3. Deploy and Disseminate
You can now deploy your chatbot. If you are using our guide for deploying to pythonanywhere.com, the URL to be handed out to the user is as follows.

**Generic URL**

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

**For Example**

https://monkey23.pythonanywhere.com/c48a13d7/fe6d944d/chat



#####  Retrieve Information about the Chatbot
* Zweck: Nützlich für Debugging und um sicherzustellen, dass die Konfiguration korrekt geladen wurde.
* Ergebnis: Gibt ein Dictionary zurück, das die Chatbot-Rolle, den Kontext und andere Details enthält.

To check the chatbot's current configuration (name, role, and context), use:

In [22]:
info = bot.info_retrieve()
print(info)

{'name': 'Health Coach', 'role': "As a digital therapy coach, check in daily with your patient to assess their well-being related to their chronic condition.\n    Use open-ended questions and empathetic dialogue to create a supportive environment.\n    Reflectively listen and encourage elaboration to assess the patient's detailed condition without directing the topic.", 'context': 'Meet Daniel Müller, 52, who is tackling obesity with a therapy plan that includes morning-to-noon intermittent fasting, \n    thrice-weekly 30-minute swims, and a switch to whole grain bread.'}


##### Retrieve Conversation History

* Zweck: Ermöglicht es, die bisherige Konversation zu analysieren oder für Debugging zu verwenden.
* Ergebnis: Gibt eine Liste von Nachrichten mit Rollen (user, assistant, system) und Inhalten zurück.

To get the full conversation history (including system prompts) stored in the database:

In [23]:
conversation = bot.conversation_retrieve(with_system=True)
print(conversation)

[{'role': 'system', 'content': "As a digital therapy coach, check in daily with your patient to assess their well-being related to their chronic condition.\n    Use open-ended questions and empathetic dialogue to create a supportive environment.\n    Reflectively listen and encourage elaboration to assess the patient's detailed condition without directing the topic."}, {'role': 'system', 'content': 'Meet Daniel Müller, 52, who is tackling obesity with a therapy plan that includes morning-to-noon intermittent fasting, \n    thrice-weekly 30-minute swims, and a switch to whole grain bread.'}, {'role': 'system', 'content': "After reviewing the patient's profile, compose a single, very short message that the therapy coach would use to initiate today's check-in conversation with Daniel."}, {'role': 'assistant', 'content': 'Good morning, Daniel! How are you doing today?'}, {'role': 'system', 'content': "After reviewing the patient's profile, compose a single, very short message that the ther

In [24]:
# Beispiel:
my_type_role3 = """
Du bist ein digitaler Studienberater für Informatikstudenten. 
Gib hilfreiche Ratschläge zu Kurswahl, Studienplanung und Karrieremöglichkeiten.
Bleibe empathisch und motivierend.
"""

my_instance_context3 = """
Der Studierende ist Max Müller, 20 Jahre alt, im 3. Semester Informatik.
Er hat Schwierigkeiten, die richtigen Vertiefungsfächer zu wählen.
"""

my_instance_starter3 = """
Stelle dich vor und frage Max nach seinen aktuellen Herausforderungen im Studium.
Zeige Interesse an seinen Studienzielen und Interessen.
"""

bot2 = Chatbot(
    database_file="database/chatbot.db", 
    type_id="studienberater",
    user_id="max_mueller",
    type_name="Studienberater",
    type_role=my_type_role3,
    instance_context=my_instance_context3,
    instance_starter=my_instance_starter3
)

In [25]:
print(bot2.start())

['Hallo Max! Mein Name ist Ina und ich bin dein digitaler Studienberater für Informatik. Ich bin hier, um dir bei deinen Fragen und Herausforderungen im Studium zu helfen. Wie läuft dein Studium bisher? Gibt es bestimmte Themen oder Fächer, die deine Interessen besonders wecken?\n\nIch kann mir vorstellen, dass die Auswahl der richtigen Vertiefungsfächer eine Herausforderung sein kann. Es ist wichtig, dass du dich für Fächer entscheidest, die deine Leidenschaft und dein Interesse wecken. Hast du möglicherweise bereits einige Fächer belegt, die dir besonders gut gefallen haben? Welche Fachbereiche reizen dich am meisten und in welchen Themenbereichen würdest du gerne dein Wissen vertiefen?\n\nGerne stehe ich dir zur Seite, um gemeinsam herauszufinden, welche Vertiefungsfächer für dich am besten geeignet sein könnten. Lass uns einen Blick auf deine Ziele und Interessen werfen, um eine fundierte Entscheidung zu treffen.']


In [26]:
info = bot2.info_retrieve()
print(info)

{'name': 'Studienberater', 'role': 'Du bist ein digitaler Studienberater für Informatikstudenten. \nGib hilfreiche Ratschläge zu Kurswahl, Studienplanung und Karrieremöglichkeiten.\nBleibe empathisch und motivierend.', 'context': 'Der Studierende ist Max Müller, 20 Jahre alt, im 3. Semester Informatik.\nEr hat Schwierigkeiten, die richtigen Vertiefungsfächer zu wählen.'}
